# Task 2
## Stock Prices Time Series Prediction
### Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

### Read data into memory

In [2]:
data = pd.read_csv('sp500.csv')
data = data.drop(['Unnamed: 0'], axis = 1)

### Correlate data

In [3]:
correlation_matrix = data.corr()

### Find Highest correlation
Get the single column with the highest correlation

In [4]:
sp_correlations = np.array(correlation_matrix[['SP500']])[1:]
correlation_index = np.argmax(sp_correlations) + 1

### Extract The Desired Dataset

In [ ]:
desired_stock = data.iloc[:, correlation_index]

input_set_1d  = np.array(desired_stock)
output_set_1d = np.array(desired_stock.shift(-5).dropna())

input_set  = [] # Process input_set_1d
output_set = [] # Process output_set_1d

train_ratio = 0.8
test_ratio  = 0.2
valid_ratio = 0.1 # Will be used in the MLPClassifier to take 10% of the training data as validation

training_input,testing_input,training_target,testing_target = train_test_split(input_set,output_set,
                                                                               test_size=test_ratio,
                                                                               random_state=1)

### Create the Linear regressor
Two multilayer perceptrons. One using a stochastic gradient descent optimizer, and the other using an adam optimizer.

Other than the optimizer, all other parameters will remain consistent for both models.

#### Scoring Metric Used:
*The mean suared error will be used as a scoring metric. The lower the error, the better the optimizer!*

#### Regularization Technique Used:
*L2 regularization was used to avoid overfitting*

In [ ]:
hidden_layer        = (512, 512, 512) # Hidden layers and nodes, per layer
regularization_rate = 0.00001         # Rate for L2 regularization
learning_rate_start = 0.01            # Initial learning rate
learning_rate_mode  = 'adaptive'      # Mode for changing learning rate
batch_size          = 128             # Size of batch for one update
tolerance           = 1e-6            # Tolerance level for regressor
max_iteration       = 1000            # Maximum number of iterations before stopping
no_change_tolerance = 10              # Stop if no significant change happens in this number of iterations

# n_iter_no_change=10

model_adm = MLPRegressor(hidden_layer_sizes=hidden_layer, solver='adam', alpha=regularization_rate,
                          batch_size=batch_size, learning_rate=learning_rate_mode,
                          learning_rate_init=learning_rate_start, max_iter=max_iteration, tol=tolerance,
                          verbose=True, early_stopping=True, validation_fraction=valid_ratio,
                          n_iter_no_change=no_change_tolerance)
model_sgd = MLPRegressor(hidden_layer_sizes=hidden_layer, solver='sgd', alpha=regularization_rate,
                          batch_size=batch_size, learning_rate=learning_rate_mode,
                          learning_rate_init=learning_rate_start, max_iter=max_iteration, tol=tolerance,
                          verbose=True, early_stopping=True, validation_fraction=valid_ratio,
                          n_iter_no_change=no_change_tolerance)